### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Esophageal_Cancer/GSE131027'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # Based on the provided series summary and title

# Determine variable availability and set corresponding keys
if "cancer: Oesophageal cancer" in sample_characteristics[1]:
    trait_row = 1

if any("age:" in item for item in sample_characteristics.get(5, [])):
    age_row = 5

if any("gender:" in item for item in sample_characteristics.get(6, [])):
    gender_row = 6

# Define conversion functions
def convert_trait(value):
    if "Oesophageal cancer" in value:
        return 1
    elif "cancer:" in value:
        return 0
    else:
        return None

def convert_age(value):
    try:
        age = int(value.split(":")[1].strip())
        return age
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender_value = value.split(":")[1].strip()
    if gender_value.lower() == 'female':
        return 0
    elif gender_value.lower() == 'male':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE131027', './preprocessed/Esophageal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Esophageal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Esophageal_Cancer/trait_data/GSE131027.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
